# Fine Tuning a GPT model by [OpenAI](https://openai.com/)

In this process i will go through the process of training `gpt-3.5-turbo-1106` model, but you can use any model like `gpt-3.5-turbo-0613` or the newly `gpt-4-0613` model. Go to OpenAI's web page and sign-in get yourself an API Key. 

![openai](../assets/images/openai.jpg)

Photo by <a href="https://unsplash.com/@jupp?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Jonathan Kemper</a> on <a href="https://unsplash.com/photos/a-close-up-of-a-computer-screen-with-a-purple-background-N8AYH8R2rWQ?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a>
  


Fine-tuning a GPT (Generative Pre-trained Transformer) model involves training an already pre-trained model on a specific downstream task or dataset. Below, I'll provide a general overview of the fine-tuning process using the popular Hugging Face Transformers library and PyTorch. Make sure you have the required libraries installed:


### Step 1: Set Up Environment

1. **Install Dependencies:**
   Ensure that you have the necessary libraries and tools installed. This typically includes deep learning frameworks like TensorFlow or PyTorch, as well as libraries for working with natural language processing tasks.

2. **Download Pre-trained Model:**
   Obtain the pre-trained LLM model weights. For GPT-3.5, you might use models from the OpenAI API or download models available from the OpenAI website.

In [ ]:
# Installing dependencies
! pip install openai torch transformers

### Step 2: Prepare Data

3. **Dataset Preparation:**
   Collect or prepare a dataset relevant to your specific task. Make sure it is formatted appropriately and has a clear structure. Divide the data into training, validation, and test sets.

4. **Data Preprocessing:**
   Tokenize and preprocess your text data to match the format used by the pre-trained model. Ensure that the input format aligns with the model's requirements, including tokenization and special tokens.

Note:

    To train a Language Model (LLM) using this dataset, the model learns the patterns and relationships within the text data. The dataset contains labeled examples of user queries and the corresponding responses from the assistant. The model learns to generate appropriate responses given user inputs.

    The training process involves feeding the input sequences (user messages) and target sequences (assistant messages) into the model. The model adjusts its parameters to minimize the difference between its predictions and the actual responses. This process is repeated iteratively until the model achieves a satisfactory level of performance.

    In the case of a chatbot like MedBot, the goal is for the trained model to generate helpful and accurate responses to a wide range of user queries related to medical information. The model learns to understand the context of the conversation, identify the user's intent, and provide relevant information or assistance.

    It's important to note that the success of the model depends on the quality and diversity of the training data. Including a variety of topics and user queries helps the model generalize well to new and unseen inputs. The dataset should also be representative of the types of interactions the chatbot is expected to handle in real-world scenarios.

In [ ]:
# This is example format 01.
{"messages": [
    {"role": "system", "content": "MedBot is a medical chatbot that provides information and assistance."},
    {"role": "user", "content": "What are the symptoms of diabetes?"},
    {"role": "assistant", "content": "Common symptoms of diabetes include increased thirst, frequent urination, and unexplained weight loss."},
]}
{"messages": [
    {"role": "system", "content": "MedBot is a medical chatbot that provides information and assistance."},
    {"role": "user", "content": "Can you explain how vaccines work?"},
    {"role": "assistant", "content": "Vaccines stimulate the immune system to recognize and fight specific pathogens, helping prevent infectious diseases."},
]}
{"messages": [
    {"role": "system", "content": "MedBot is a medical chatbot that provides information and assistance."},
    {"role": "user", "content": "What is the recommended daily intake of vitamin C?"},
    {"role": "assistant", "content": "The recommended daily intake of vitamin C varies by age and gender, but for adults, it's generally around 90mg for men and 75mg for women."},
]}

The given dataset represents interactions with a medical chatbot named MedBot. Each interaction is structured as a series of messages between different roles: "system," "user," and "assistant." The content of each message provides information about the conversation.

Here's a breakdown of each conversation:

1. **Conversation 1:**
   - System message: Introduction of MedBot.
   - User message: Inquiry about the symptoms of diabetes.
   - Assistant message: Provides information about common symptoms of diabetes.

2. **Conversation 2:**
   - System message: Introduction of MedBot.
   - User message: Request for an explanation of how vaccines work.
   - Assistant message: Provides information on how vaccines stimulate the immune system.

3. **Conversation 3:**
   - System message: Introduction of MedBot.
   - User message: Inquiry about the recommended daily intake of vitamin C.
   - Assistant message: Provides information about the recommended daily intake of vitamin C.

In [ ]:
# This is example format 02.
{
    "prompt": "Describe the symptoms and medical history of a patient with hypertension.",
    "completion": "A patient presents with elevated blood pressure, consistently measuring above 140/90 mm Hg. The individual has a history of sedentary lifestyle, high-sodium diet, and a family history of hypertension. Additionally, the patient reports occasional headaches and dizziness. Provide recommendations for lifestyle modifications and prescribe antihypertensive medication if necessary."
}

{
    "prompt": "Explain the pathophysiology of diabetes mellitus.",
    "completion": "Diabetes mellitus is a metabolic disorder characterized by hyperglycemia resulting from defects in insulin secretion, insulin action, or both. In type 1 diabetes, there is an autoimmune destruction of pancreatic beta cells leading to absolute insulin deficiency. Type 2 diabetes involves insulin resistance and relative insulin deficiency. Describe the mechanisms involved and discuss the complications associated with uncontrolled diabetes, emphasizing the importance of glycemic control through medication, diet, and exercise."
}

{
    "prompt": "Outline the procedure for a routine physical examination.",
    "completion": "A routine physical examination involves a comprehensive assessment of the patient's overall health. Begin with a thorough medical history, followed by vital signs measurement, inspection, palpation, percussion, and auscultation. Perform a head-to-toe examination, including the cardiovascular, respiratory, gastrointestinal, musculoskeletal, and neurological systems. Document any abnormalities, discuss findings with the patient, and recommend further diagnostic tests or interventions as needed."
}


Let's break down how this dataset can be used to train a Language Model (LM):

1. **Prompt for Hypertension:**
   - **Prompt:** "Describe the symptoms and medical history of a patient with hypertension."
   - **Completion:** "A patient presents with elevated blood pressure, consistently measuring above 140/90 mm Hg. The individual has a history of sedentary lifestyle, high-sodium diet, and a family history of hypertension. Additionally, the patient reports occasional headaches and dizziness. Provide recommendations for lifestyle modifications and prescribe antihypertensive medication if necessary."

   **Training Usage:** This pair helps the model understand the symptoms, risk factors, and management of hypertension. The model learns the context of hypertension and how to provide recommendations based on a patient's symptoms and medical history.

2. **Prompt for Diabetes Mellitus:**
   - **Prompt:** "Explain the pathophysiology of diabetes mellitus."
   - **Completion:** "Diabetes mellitus is a metabolic disorder characterized by hyperglycemia resulting from defects in insulin secretion, insulin action, or both. In type 1 diabetes, there is an autoimmune destruction of pancreatic beta cells leading to absolute insulin deficiency. Type 2 diabetes involves insulin resistance and relative insulin deficiency. Describe the mechanisms involved and discuss the complications associated with uncontrolled diabetes, emphasizing the importance of glycemic control through medication, diet, and exercise."

   **Training Usage:** This pair teaches the model about the pathophysiology of diabetes mellitus, differentiating between type 1 and type 2 diabetes. It also covers complications and emphasizes the importance of glycemic control through various means.

3. **Prompt for Routine Physical Examination:**
   - **Prompt:** "Outline the procedure for a routine physical examination."
   - **Completion:** "A routine physical examination involves a comprehensive assessment of the patient's overall health. Begin with a thorough medical history, followed by vital signs measurement, inspection, palpation, percussion, and auscultation. Perform a head-to-toe examination, including the cardiovascular, respiratory, gastrointestinal, musculoskeletal, and neurological systems. Document any abnormalities, discuss findings with the patient, and recommend further diagnostic tests or interventions as needed."

   **Training Usage:** This pair guides the model in understanding the steps involved in a routine physical examination. It covers the different aspects of the examination process, helping the model generate coherent and informative responses when asked about routine physical examinations.

In summary, this dataset provides a diverse set of prompts and responses related to medical topics, allowing the language model to learn and generalize information about hypertension, diabetes mellitus, and routine physical examinations. The training process involves exposing the model to various examples to improve its ability to generate relevant and accurate information in response to similar queries.

### Step 3: Model Configuration

5. **Model Architecture:**
   Define the architecture of your fine-tuned model. Typically, you will use the architecture of the pre-trained LLM and adapt it for your specific task. This may involve adding task-specific layers on top of the pre-trained model.

To use the file with a fine-tuning job, you must upload it via the Files API after the data has been verified:

In [ ]:
from openai import OpenAI

client = OpenAI()

client.files.create(
  file=open("dataset.json", "rb"),
  purpose="fine-tune"
)

The processing of the file could take some time after you upload it. You can still start a fine-tuning job while the file is processing, but it won't begin until the processing is finished.

### Step 4: Fine-tuning Process

6. **Loss Function:**
   Choose or create a loss function appropriate for your task. This function will be used to measure the difference between the model's predictions and the actual target values during training.

7. **Optimizer:**
   Select an optimizer (e.g., Adam, SGD) to minimize the loss during training. Tune hyperparameters such as learning rate to achieve better convergence.

8. **Training Loop:**
   Implement the training loop, where you feed batches of your training data into the model, calculate the loss, and update the model's weights using backpropagation.

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-abc123",
  model="gpt-3.5-turbo"
)

The model name in this case is model, which can be any of the following: gpt-3.5-turbo, babbage-002, davinci-002, or an already-fine-tuned model. The file ID returned by the OpenAI API upon uploading the training file is training_file. You can use the suffix parameter to change the name of your fine-tuned model.

Please refer to the API specification for fine-tuning if you would like to set any additional parameters, such as the validation_file or hyperparameters.

Once you begin a fine-tuning task, it might take some time to finish. Depending on the model and dataset size, training a model can take minutes or hours, and your job might be queued behind other jobs in our system. The user who made the fine-tuning job will get after the model training is finished.

In [ ]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-abc123")

# Cancel a job
client.fine_tuning.jobs.cancel("ftjob-abc123")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

### Step 5: Monitor and Adjust

9. **Validation:**
   Periodically evaluate your model on the validation set to monitor its performance. This helps prevent overfitting and allows you to adjust hyperparameters or stop training if necessary.

10. **Fine-tuning Parameters:**
    Experiment with different hyperparameters, such as learning rate and batch size, to optimize the model's performance on your specific task.

### Step 6: Evaluation

11. **Test Set Evaluation:**
    After training, assess the model's performance on the test set to gauge its generalization ability to unseen data.

In [ ]:
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo:my-org:custom_suffix:id",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)

OpenAI offer the following training metrics, which are calculated during the training process: test loss, test token accuracy, training loss, and training loss. The purpose of these statistics is to serve as a sanity check that the training process went well (accuracy of tokens should rise, loss should drop). You can see an event object with the following helpful metrics while a fine-tuning job is active:

In [ ]:
{
    "object": "fine_tuning.job.event",
    "id": "ftevent-abc-123",
    "created_at": 1693582679,
    "level": "info",
    "message": "Step 100/100: training loss=0.00",
    "data": {
        "step": 100,
        "train_loss": 1.805623287509661e-5,
        "train_mean_token_accuracy": 1.0
    },
    "type": "metrics"
}

In conclusion, finishing a model's fine-tuning for medical applications is an important step, but it's only the start of a longer and more involved process. To guarantee that the model is deployed, monitored, and improved upon in a medical setting, a number of crucial actions must be taken.

First and foremost, it is imperative to thoroughly validate and test the refined model on a variety of representative datasets. This step aids in evaluating the robustness and generalizability of the model across various patient populations and scenarios. Thorough assessment is necessary to guarantee the model's dependability and efficiency in practical settings.

The model should be deployed carefully and morally after it has been validated. Working together with regulatory agencies, healthcare providers, and other pertinent parties is crucial to a smooth integration of the model into clinical workflows. To win the trust and acceptance of the medical community, it is imperative that the model's capabilities, limitations, and potential risks are transparently documented.

It is imperative to conduct ongoing monitoring and evaluation of the model's performance in real-world scenarios. It is important to regularly update and improve the system in response to user feedback, fresh data, and new developments in medicine. Given the dynamic nature of medical information, this adaptive approach guarantees that the model will continue to be accurate and relevant over time.

Furthermore, it is critical to uphold adherence to healthcare policies, patient privacy laws, and ethical standards. An essential component of the model's long-term viability and acceptance in the medical community is protecting patient data and guaranteeing its responsible and secure use.

Ultimately, the development of a successful feedback loop depends on promoting cooperation and communication amongst data scientists, medical professionals, and other stakeholders. Establishing regular channels of communication can aid in addressing new issues, improving the model, and enhancing its clinical utility.

To sum up, following the process of fine-tuning a medical model, it is imperative to engage in ongoing validation, ethical deployment, continuous monitoring, and collaboration to guarantee the model's efficacy, safety, and sustained success in improving healthcare outcomes.

Credit: OpenAI <br/>
Link: [OpenAI.com](https://platform.openai.com/docs/guides/fine-tuning)